## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-25-01-16-34 +0000,nypost,Michelle Obama accused of taking Ozempic after...,https://nypost.com/2025/11/24/entertainment/mi...
1,2025-11-25-01-13-07 +0000,nypost,Drugs sneaked into Ohio prison soaked into the...,https://nypost.com/2025/11/24/us-news/drugs-sn...
2,2025-11-25-01-07-27 +0000,nyt,Schumer Faces Pushback From ‘Fight Club’ Group...,https://www.nytimes.com/2025/11/24/us/politics...
3,2025-11-25-01-06-36 +0000,bbc,Zelensky welcomes amendments to proposed peace...,https://www.bbc.com/news/articles/cpd6qd10l94o...
4,2025-11-25-01-06-10 +0000,nyt,Trump Orders Construction of A.I. Platform to ...,https://www.nytimes.com/2025/11/24/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2364/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
36,trump,40
111,ukraine,22
34,peace,17
35,plan,17
9,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
124,2025-11-24-20-17-58 +0000,nypost,"US, Ukraine agree on new 19-point peace plan t...",https://nypost.com/2025/11/24/world-news/us-uk...,129
278,2025-11-24-07-31-06 +0000,nyt,"Ukraine, U.S. Meet About Trump’s Peace Plan to...",https://www.nytimes.com/2025/11/23/world/europ...,122
11,2025-11-25-00-42-04 +0000,nyt,What to Know About Trump’s Peace Plan for Russ...,https://www.nytimes.com/2025/11/24/us/politics...,109
277,2025-11-24-07-32-21 +0000,nyt,What Is Trump’s 28-Point Plan to End Russia’s ...,https://www.nytimes.com/2025/11/22/world/europ...,106
64,2025-11-24-22-43-00 +0000,wsj,Ukraine Peace-Plan Negotiators Leave Thorniest...,https://www.wsj.com/world/europe/big-gaps-stil...,104


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
124,129,2025-11-24-20-17-58 +0000,nypost,"US, Ukraine agree on new 19-point peace plan t...",https://nypost.com/2025/11/24/world-news/us-uk...
162,65,2025-11-24-17-49-48 +0000,nypost,Judge dismisses cases against ex-FBI Director ...,https://nypost.com/2025/11/24/us-news/judge-di...
149,52,2025-11-24-18-33-00 +0000,wsj,President Trump’s advisers are discussing a pr...,https://www.wsj.com/politics/policy/trump-advi...
165,38,2025-11-24-17-44-49 +0000,nypost,"UK teen claims people threw urine, glue at him...",https://nypost.com/2025/11/24/world-news/uk-te...
71,35,2025-11-24-22-22-05 +0000,wapo,"In Gulf of America case, AP renews legal fight...",https://www.washingtonpost.com/business/2025/1...
159,32,2025-11-24-18-04-19 +0000,nypost,Putin believes Russia-friendly US cease-fire p...,https://nypost.com/2025/11/24/world-news/russi...
135,31,2025-11-24-19-33-58 +0000,bbc,Pentagon to review Senator Mark Kelly miscondu...,https://www.bbc.com/news/articles/cqxqwrlp7r0o...
157,30,2025-11-24-18-08-00 +0000,wsj,The Federal Reserve is struggling to persuade ...,https://www.wsj.com/economy/central-banking/th...
82,30,2025-11-24-21-59-32 +0000,nypost,Trump accepts Xi’s invite to Beijing and calls...,https://nypost.com/2025/11/24/us-news/trump-ac...
41,28,2025-11-24-23-27-47 +0000,nypost,Anti-ICE protesters clog up California Home De...,https://nypost.com/2025/11/24/us-news/anti-ice...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
